# Advanced Tool Usage: Parallel Calls and Iterations

## Overview

This notebook delves into advanced configurations for tool usage within AutoGen AgentChat, specifically focusing on:

- **Parallel Tool Calls**: How agents can execute multiple tools concurrently when supported by the underlying language model.
- **Tool Iterations**: Configuring agents to perform multiple rounds of tool calls until a task is complete or a maximum iteration limit is reached.

These features provide greater flexibility and efficiency for agents tackling complex tasks that require multiple external interactions.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

## Parallel Tool Calls

Some advanced language models (like OpenAI's GPT-4o) can generate multiple tool calls in a single response. By default, `AssistantAgent` will execute these tool calls in parallel.

You might want to disable parallel tool calls if your tools have side effects that could interfere with each other, or if you need consistent behavior across models that don't support parallel calls.

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# Define two mock tools
async def get_weather(city: str) -> str:
    """Gets the current weather for a specified city."""
    await asyncio.sleep(1) # Simulate network delay
    if city.lower() == "london":
        return "The weather in London is cloudy with a temperature of 15°C."
    elif city.lower() == "paris":
        return "The weather in Paris is sunny with a temperature of 22°C."
    return f"Could not find weather for {city}."

async def get_time(city: str) -> str:
    """Gets the current time for a specified city."""
    await asyncio.sleep(0.5) # Simulate network delay
    if city.lower() == "london":
        return "The current time in London is 10:00 AM GMT."
    elif city.lower() == "paris":
        return "The current time in Paris is 11:00 AM CEST."
    return f"Could not find time for {city}."

async def run_parallel_tool_example():
    # Agent with parallel tool calls enabled (default for supporting models)
    model_client_parallel = OpenAIChatCompletionClient(model="gpt-4o-mini")
    agent_parallel = AssistantAgent(
        name="parallel_assistant",
        model_client=model_client_parallel,
        tools=[get_weather, get_time],
        system_message="You are an assistant that can get weather and time for cities. Use both tools if needed."
    )

    print("\n🔄 Task (Parallel Enabled): What is the weather and time in London and Paris?")
    print("-" * 40)
    await Console(agent_parallel.run_stream(task="What is the weather and time in London and Paris?"))

    # Agent with parallel tool calls disabled
    model_client_no_parallel = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        parallel_tool_calls=False # Explicitly disable parallel calls
    )
    agent_no_parallel = AssistantAgent(
        name="sequential_assistant",
        model_client=model_client_no_parallel,
        tools=[get_weather, get_time],
        system_message="You are an assistant that can get weather and time for cities. Use both tools if needed."
    )

    print("\n🔄 Task (Parallel Disabled): What is the weather and time in London and Paris?")
    print("-" * 40)
    await Console(agent_no_parallel.run_stream(task="What is the weather and time in London and Paris?"))

    await model_client_parallel.close()
    await model_client_no_parallel.close()

await run_parallel_tool_example()


🔄 Task (Parallel Enabled): What is the weather and time in London and Paris?
----------------------------------------
---------- TextMessage (user) ----------
What is the weather and time in London and Paris?
---------- ToolCallRequestEvent (parallel_assistant) ----------
[FunctionCall(id='call_cMsdlMuPHU6z25jayzDKM4VW', arguments='{"city": "London"}', name='get_weather'), FunctionCall(id='call_wVHQoli0rD6ONZkXrh90kDRZ', arguments='{"city": "London"}', name='get_time'), FunctionCall(id='call_zMM8aGMxg9rxpAAf33k0rxdx', arguments='{"city": "Paris"}', name='get_weather'), FunctionCall(id='call_D6zznw1JvoassWGOizpUQOoe', arguments='{"city": "Paris"}', name='get_time')]
---------- ToolCallExecutionEvent (parallel_assistant) ----------
[FunctionExecutionResult(content='The weather in London is cloudy with a temperature of 15°C.', name='get_weather', call_id='call_cMsdlMuPHU6z25jayzDKM4VW', is_error=False), FunctionExecutionResult(content='The current time in London is 10:00 AM GMT.', name='

## Tool Iterations

By default, an `AssistantAgent` performs at most one tool iteration (one model call followed by one or more parallel tool calls). For tasks requiring multiple steps of tool interaction, you can configure the agent to execute multiple iterations using the `max_tool_iterations` parameter.

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# A mock tool that simulates a multi-step process
call_count = 0
async def get_next_step_data(step: int) -> str:
    """Fetches data for a specific step in a multi-step process."""
    global call_count
    call_count += 1
    if step == 1:
        return "Initial data: User wants to process file A."
    elif step == 2:
        return "Processing file A complete. Next, process file B."
    elif step == 3:
        return "Processing file B complete. All files processed."
    return "No more steps."

async def run_tool_iterations_example():
    global call_count
    call_count = 0 # Reset for demonstration

    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    agent_loop = AssistantAgent(
        name="iterative_assistant",
        model_client=model_client,
        tools=[get_next_step_data],
        system_message=(
            "You are an assistant that processes data in multiple steps. "
            "Call 'get_next_step_data' with the current step number (starting from 1). "
            "Continue calling the tool until all steps are complete. "
            "Report the final status when done."
        ),
        max_tool_iterations=5 # Allow up to 5 tool calls in a loop
    )

    print("\n🔄 Task: Process all data steps.")
    print("-" * 40)
    await Console(agent_loop.run_stream(task="Start data processing."))

    print(f"\nTotal tool calls made: {call_count}")
    await model_client.close()

await run_tool_iterations_example()


🔄 Task: Process all data steps.
----------------------------------------
---------- TextMessage (user) ----------
Start data processing.
---------- ToolCallRequestEvent (iterative_assistant) ----------
[FunctionCall(id='call_eWznPrWKuQNgfEQCtP5HkFMD', arguments='{"step":1}', name='get_next_step_data')]
---------- ToolCallExecutionEvent (iterative_assistant) ----------
[FunctionExecutionResult(content='Initial data: User wants to process file A.', name='get_next_step_data', call_id='call_eWznPrWKuQNgfEQCtP5HkFMD', is_error=False)]
---------- ToolCallRequestEvent (iterative_assistant) ----------
[FunctionCall(id='call_j8t4C9OscftWS75b27glbLRB', arguments='{"step":2}', name='get_next_step_data')]
---------- ToolCallExecutionEvent (iterative_assistant) ----------
[FunctionExecutionResult(content='Processing file A complete. Next, process file B.', name='get_next_step_data', call_id='call_j8t4C9OscftWS75b27glbLRB', is_error=False)]
---------- ToolCallRequestEvent (iterative_assistant) ----

## Next Steps

Experiment further with advanced tool usage:

1.  **Complex Parallel Scenarios**: Design a scenario where parallel tool calls are genuinely beneficial (e.g., fetching data from multiple independent APIs).
2.  **Conditional Iterations**: Implement a tool that returns a specific signal (e.g., a boolean flag) to tell the agent when to stop iterating, rather than relying solely on `max_tool_iterations`.
3.  **Error Handling**: Explore how agents handle errors during tool execution in iterative or parallel scenarios.